In [155]:
from __future__ import division

import numpy as np
import pandas as pd
import json
import codecs
import os
import re

from tqdm import tqdm_notebook
from datetime import datetime
from string import punctuation

Посмотрим на одну из строчек в JSON-файле: считаем ее с помощью библиотеки json. Эта строчка соответствует [7-ой статье](https://habrahabr.ru/post/7/) на Хабре.

In [144]:
!head -1 ../../data/data.hw8/train.json > ../../data/data.hw8/train1.json
with codecs.open('../../data/data.hw8/train1.json', 'r', 'utf-8') as inp_json:
    first_json = json.load(inp_json)
os.remove('../../data/data.hw8/train1.json')

target = pd.read_csv('../../data/data.hw8/train_target.csv', index_col='url')
sub = pd.read_csv('../../data/data.hw8/sample_submission.csv', index_col='url')

In [145]:
print(json.dumps(first_json, indent=4, sort_keys=False, ensure_ascii=False))

{
    "domain": "habrahabr.ru", 
    "_timestamp": 1493192186.0903192, 
    "meta_tags": {
        "msapplication-TileColor": "#FFFFFF", 
        "application-name": "Хабрахабр", 
        "fb:app_id": "444736788986613", 
        "twitter:site": "@habrahabr", 
        "al:windows_phone:app_name": "Habrahabr", 
        "og:url": "https://habrahabr.ru/post/7/", 
        "pocket-site-verification": "ed24b2b9721edf0a282c5b4a3232c4", 
        "og:type": "article", 
        "al:android:package": "ru.habrahabr", 
        "viewport": "width=1024", 
        "og:title": "Самопроизвольное разлогинивание", 
        "description": "У меня такое ощущение, что logout время от времени происходит самопроизвольно, несмотря на то, что чекбокс про логине включен.\r\n\r\nВозможно, это происходит при смене IP-адреса, но я не уверен.", 
        "robots": "noindex", 
        "msapplication-TileImage": "mstile-144x144.png", 
        "al:android:app_name": "Habrahabr", 
        "og:image": "https://habrahabr.ru/

Видим 16 полей, перечислим некоторые из них:
- _id, url - URL статьи
- published – время публикации статьи
- domain – сайт (например, habrahahbr.ru или geektimes.ru)
- title – название статьи
- content – текст статьи
- hubs - перечисление хабов, к которым относится статья
- tags – теги статьи
- author – автор статьи, его ник и ссылка на профиль

Сформируйте обучающую выборку для Vowpal Wabbit, выберите признаки title, tags, domain, flow, author, и hubs из JSON-файла. От самого текста для начала просто возьмем его длину: постройте признак content_len – длина текста в миллионах символов. Также постройте признаки: час и месяц публикации статьи. Еще, конечно же, возьмите ответы на обучающей выборке из `train_target`.

In [197]:
def to_vw_format(document, label=1):
    r = str(label) # start with label, add all string-like features
    
    r += ' |title '  + document['title'].lower().translate(dict.fromkeys(ord(c) for c in punctuation)) + \
         ' |xtags '  + ' '.join(document['tags']) + \
         ' |domain ' + document['domain'] + \
         ' |author ' + (document['author']['nickname'] or 'None') + \
         ' |hubs '   + document['hubs'][0]['title']
            
    r = r.replace(':', '')        # remove all colons
    r = r.replace(u'\u2028', '')  # remove bullet symbol
    
    # add numerical features
    pub = datetime.strptime(document['published']['$date'], '%Y-%m-%dT%H:%M:%S.%fZ')
    r += ' |num' + \
         ' content_len:' + str( round(len(document['content']) / 10**6, 2 )) + \
         ' month:'      + str(pub.month) + \
         ' hour:'       + str(pub.hour)
    
    # add my features
    r += ' |mynum' + \
         ' link:' + str(document['content'].count('<a href=')) + \
         ' user:' + str(document['content'].count('user_link')) + \
         ' image:' + str(document['content'].count('img src')) + \
         ' video:' + str(document['content'].count('iframe width')) + \
         ' day:' + pub.strftime("%-m") + \
         ' weekday:' + str(pub.weekday()) + \
         ' weekend:' + str(int(pub.weekday() > 4)) + \
         ' year:' + str(pub.year)
            
    r += ' |weekday ' + pub.strftime("%a")

    description = (document['meta_tags']['og:description'] or 'None')
    r += ' |description ' + ' '.join(re.findall(r'(?u)\w{3,}', description.lower()))
    
    return r + '\n'

# Demo on first record

In [198]:
print to_vw_format(first_json, target.iloc[0]['target'])

0.693147 |title самопроизвольное разлогинивание |xtags логин login |domain habrahabr.ru |author @ptitov |hubs Хабрахабр |num content_len:0.0 month:7 hour:1 |mynum link:0 user:0 image:0 video:0 day:7 weekday:5 weekend:1 year:2006 |weekday Sat |description меня такое ощущение что logout время времени происходит самопроизвольно несмотря что чекбокс про логине включен возможно это происходит при



# Process train and test

In [199]:
with open('../../data/data.hw8/train.json') as inp_json, \
      codecs.open('../../data/data.hw8/habr_train.vw', 'w', 'utf-8') as out_vw:
    for i, line in enumerate(tqdm_notebook(inp_json, total=len(target))):
        data_json = json.loads(line)
        out_vw.write(to_vw_format(data_json, target.iloc[i]['target']))
        
with codecs.open('../../data/data.hw8/habr_train.vw', 'r', 'utf-8') as f:
    for i, _ in enumerate(f):
        pass

assert i+1 == len(target), 'WARNING: Length of vw data is ' + str(i+1) + \
                                 ' and length of target is ' + str(len(target))
    
print str(i+1) + ' lines in train set'


120000 lines in train set


In [200]:
!head -2 ../../data/data.hw8/habr_train.vw

0.693147 |title самопроизвольное разлогинивание |xtags логин login |domain habrahabr.ru |author @ptitov |hubs Хабрахабр |num content_len:0.0 month:7 hour:1 |mynum link:0 user:0 image:0 video:0 day:7 weekday:5 weekend:1 year:2006 |weekday Sat |description меня такое ощущение что logout время времени происходит самопроизвольно несмотря что чекбокс про логине включен возможно это происходит при
1.098612 |title standalong cообщества против сообществ в рамках социальных сетей |xtags сообщества интернет-сообщество социальные сети нишевой бренд |domain geektimes.ru |author @AlexBruce |hubs Чёрная дыра |num content_len:0.0 month:7 hour:14 |mynum link:1 user:0 image:0 video:0 day:7 weekday:5 weekend:1 year:2006 |weekday Sat |description вот тут подумал смотря скажем комби зачем надо создавать социальную сеть чтобы потом там формировать сообщества ведь сразу возникает вопрос откуда


In [201]:
with open('../../data/data.hw8/test.json') as inp_json, \
     codecs.open('../../data/data.hw8/habr_test.vw', 'w', 'utf-8') as out_vw:
    for line in tqdm_notebook(inp_json, total=len(sub)):
        data_json = json.loads(line)
        out_vw.write(to_vw_format(data_json))
        
with codecs.open('../../data/data.hw8/habr_test.vw', 'r', 'utf-8') as f:
    for i, _ in enumerate(f):
        pass
    
assert i+1 == len(sub), 'WARNING: Length of vw data is ' + str(i+1) + \
                                 ' and length of submission is ' + str(len(sub))
    
print str(i+1) + ' lines in test set'


52913 lines in test set


In [202]:
!head -2 ../../data/data.hw8/habr_test.vw

1 |title день пи |xtags Пи Pi |domain geektimes.ru |author @Timursan |hubs Чёрная дыра |num content_len:0.0 month:3 hour:3 |mynum link:0 user:0 image:1 video:0 day:3 weekday:6 weekend:1 year:2010 |weekday Sun |description поздравляю всех днём столько вам успехов сколько знаков после запятой числе
1 |title скрипт для разбиения образов музыкальных cd на треки и конвертации в формат flac |xtags bash lossless |domain geektimes.ru |author @da3mon |hubs Чёрная дыра |num content_len:0.01 month:3 hour:0 |mynum link:0 user:0 image:0 video:0 day:3 weekday:6 weekend:1 year:2010 |weekday Sun |description здравствуйте сообщество так сложилось что просторах сети можно найти массу музыки lossless форматах flac monkey audio wavpack лично мне
